In [1]:
import sys
sys.path.append('/Users/sbhardwaj/Documents/GraphNodeClassification')

from src.models.models import GCN
from src.data.data_loader import DBLP_dataset
from src.utilities.utils import acc , recall , create_communities , create_nodeinfo
import node2vec
import torch
import os
import numpy as np
from sklearn.metrics import multilabel_confusion_matrix , ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import networkx as nx

### Selecting device

In [3]:
if torch.backends.mps.is_available():
    device = torch.device("mps")
    print("mps device found")
else:
    device = "cpu"
    print ("MPS device not found.")

mps device found


## Loading unseen data

In [32]:
dataloader_args = {
    "root":os.path.abspath("..") + "/data",
    "raw_filenames":["graph_edges.txt" , "5000_communities.txt"],
    "expt":"edge_sampling",
    "processed_filenames":["data_nc_0.pt" , "data_nc_1.pt" , "data_nc_2.pt" , "data_nc_3.pt" ]
}

dataloader_args

{'root': '/Users/sbhardwaj/Documents/GraphNodeClassification/data',
 'raw_filenames': ['graph_edges.txt', '5000_communities.txt'],
 'expt': 'edge_sampling',
 'processed_filenames': ['data_nc_0.pt',
  'data_nc_1.pt',
  'data_nc_2.pt',
  'data_nc_3.pt']}

In [33]:
d_nc = DBLP_dataset(root = dataloader_args["root"] , raw_filenames = dataloader_args["raw_filenames"] ,
                    expt = "n_communities", processed_filenames = dataloader_args["processed_filenames"])

Processing...
Done!


In [4]:
data = torch.load(f"../data/processed/data_16_200_0.5.pt").to(device)
data

Data(x=[30732, 21], edge_index=[2, 142336], y=[30732, 200], dtype=torch.float32, g=Graph with 30732 nodes and 55802 edges, train_mask=[30732], val_mask=[30732], test_mask=[30732])

## Arguments for Node2Vec embedding generation and model

In [6]:
gcn_args = {
    'device': device,
    'num_layers': 4,
    'hidden_dim': 16,
    'dropout': 0.2,
    'lr': 0.005,
    'epochs': 1000,
}

gcn_args

{'device': device(type='mps'),
 'num_layers': 4,
 'hidden_dim': 16,
 'dropout': 0.2,
 'lr': 0.005,
 'epochs': 1000}

### Embeddings generation

### Loading model weights

In [7]:
model_path = "../models/gcn_16_200_0.5_1.pt"

loaded_model = GCN(data.x.shape[1] , gcn_args['hidden_dim'] , data.y.shape[1] , gcn_args['num_layers'] , gcn_args['dropout']).to(device)
loaded_model.load_state_dict(torch.load(model_path))

<All keys matched successfully>

### Testing the output

In [8]:
y = loaded_model(data.x , data.edge_index)

loc("mps_not_equal"("(mpsFileLoc): /AppleInternal/Library/BuildRoots/4e1473ee-9f66-11ee-8daf-cedaeb4cabe2/Library/Caches/com.apple.xbs/Sources/MetalPerformanceShadersGraph/mpsgraph/MetalPerformanceShadersGraph/Core/Files/MPSGraphUtilities.mm":253:0)): error: 'anec.not_equal_zero' op Invalid configuration for the following reasons: Tensor dimensions N1D1C1H1W142336 are not within supported range, N[1-65536]D[1-16384]C[1-65536]H[1-16384]W[1-16384].
loc("mps_select"("(mpsFileLoc): /AppleInternal/Library/BuildRoots/4e1473ee-9f66-11ee-8daf-cedaeb4cabe2/Library/Caches/com.apple.xbs/Sources/MetalPerformanceShadersGraph/mpsgraph/MetalPerformanceShadersGraph/Core/Files/MPSGraphUtilities.mm":294:0)): error: 'anec.not_equal_zero' op Invalid configuration for the following reasons: Tensor dimensions N1D1C1H1W142336 are not within supported range, N[1-65536]D[1-16384]C[1-65536]H[1-16384]W[1-16384].
loc("mps_select"("(mpsFileLoc): /AppleInternal/Library/BuildRoots/4e1473ee-9f66-11ee-8daf-cedaeb4cabe

In [9]:
c = (multilabel_confusion_matrix((data.y[data.train_mask].int()).cpu() , ((y>0.5)[data.train_mask].int()).cpu()))

loc("mps_not_equal"("(mpsFileLoc): /AppleInternal/Library/BuildRoots/4e1473ee-9f66-11ee-8daf-cedaeb4cabe2/Library/Caches/com.apple.xbs/Sources/MetalPerformanceShadersGraph/mpsgraph/MetalPerformanceShadersGraph/Core/Files/MPSGraphUtilities.mm":253:0)): error: 'anec.not_equal_zero' op Invalid configuration for the following reasons: Tensor dimensions N1D1C1H1W30732 are not within supported range, N[1-65536]D[1-16384]C[1-65536]H[1-16384]W[1-16384].
loc("mps_select"("(mpsFileLoc): /AppleInternal/Library/BuildRoots/4e1473ee-9f66-11ee-8daf-cedaeb4cabe2/Library/Caches/com.apple.xbs/Sources/MetalPerformanceShadersGraph/mpsgraph/MetalPerformanceShadersGraph/Core/Files/MPSGraphUtilities.mm":294:0)): error: 'anec.not_equal_zero' op Invalid configuration for the following reasons: Tensor dimensions N1D1C1H1W30732 are not within supported range, N[1-65536]D[1-16384]C[1-65536]H[1-16384]W[1-16384].


In [10]:
recall(np.sum(c , axis=0))

0.9564424866137546

In [37]:
comms = create_communities(y , thresh=0.95)
(len(comms[35]))

23681

In [39]:
nodeinfo = create_nodeinfo(y , thresh=0.9)

In [41]:
idx = 75
print(nodeinfo[0])

[array([35.        ,  0.98201376]), array([178.        ,   0.90843707])]
